<a href="https://colab.research.google.com/github/cyberust/HumanSynergyAnalysis_JP/blob/main/HumanSynergyAnalysis_JP_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ======================================================================
#
#  人的シナジー分析システム V4 - 総合安定版
#
#  特徴:
#  - 初期の全分析機能（ROI, モンテカルロ等）を搭載
#  - V3で確立した安定的な実行パイプライン、動的モデル、I/O処理を全面適用
#
# ======================================================================

# ===============================================================
# STEP 1: ライブラリのインストール
# ===============================================================
!pip install pandas numpy scipy networkx plotly scikit-learn textblob vaderSentiment --quiet
print("✅ ライブラリのインストールが完了しました。")

# ===============================================================
# STEP 2: ライブラリのインポート
# ===============================================================
import os
import re
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from scipy.integrate import odeint
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
from google.colab import drive
import shutil
import time
import math
from itertools import combinations
print("✅ ライブラリのインポートが完了しました。")


# ===============================================================
# STEP 3: 分析パラメータの一元管理
# ===============================================================
CONFIG = {
    'data': {
        'base_path': "/content/drive/MyDrive/",
        'names': ["arale_cohen", "yanay_geva", "yasuyuki_sakane"],
        'display_names': ["Arale Cohen", "Yanay Geva", "Yasuyuki Sakane"]
    },
    'profile_analysis': {
        'experience_default': 10,
        'network_keywords': ['network', 'ecosystem', 'connections', 'board member', 'partner', 'collaboration', 'global', 'silicon valley'],
        'network_multiplier': 15,
        'skill_keywords': {
            'AI/ML': ['artificial intelligence', 'machine learning', 'deep learning', 'ai', 'ml'],
            'Technology': ['technology', 'tech', 'engineering', 'software', 'system', 'architecture'],
            'Finance': ['finance', 'fintech', 'venture capital', 'investment', 'funding', 'vc'],
            'Leadership': ['leadership', 'management', 'team', 'cto', 'ceo', 'executive'],
            'Product': ['product', 'development', 'launch', 'strategy', 'planning'],
        }
    },
    'synergy_calculation': {
        'weights': {'complementarity': 0.4, 'experience': 0.3, 'network': 0.3},
        'cultural_backgrounds': {'Arale Cohen': 'Western', 'Yanay Geva': 'Western', 'Yasuyuki Sakane': 'Eastern'},
        'diversity_bonus': 1.2,
    },
    'dynamic_model': {
        'alpha': np.array([0.4, 0.35, 0.45]), 'beta_multiplier': 0.1, 'K': np.array([12.0, 12.0, 12.0]),
        'initial_state': [1.0, 1.0, 1.0], 'time_horizon': 24
    },
    'game_theory': { 'synergy_value_multiplier': 5 },
    'prediction': {
        'monte_carlo_simulations': 1000,
        'investment_cost': 1000000,
        'revenue_scale_factor': 10000,
        'npv_discount_rate': 0.05
    },
    'output': {
        'base_path': "/content/drive/MyDrive/",
        'folder_prefix': "synergy_full_analysis_"
    }
}
print("✅ 分析パラメータの定義が完了しました。")


# ===============================================================
# STEP 4: 各種分析クラスの定義
# ===============================================================

class DataLoader:
    def __init__(self, config): self.config = config['data']
    def load_profiles(self):
        mount_point = '/content/drive'
        if os.path.exists(mount_point) and not os.path.ismount(mount_point): shutil.rmtree(mount_point)
        drive.mount(mount_point, force_remount=True)
        profiles = {}
        for name, display_name in zip(self.config['names'], self.config['display_names']):
            try:
                with open(f"{self.config['base_path']}{name}_profile.txt", 'r', encoding='utf-8') as f:
                    profiles[display_name] = f.read()
            except FileNotFoundError: profiles[display_name] = "Profile data not available."
        return profiles

class KeywordProfileAnalyzer:
    def __init__(self, config):
        self.config = config['profile_analysis']
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
    def analyze(self, profiles_dict):
        return {name: {'skills': self._extract_skills(text),
                       'experience_years': self._extract_experience(text),
                       'network_strength': self._extract_network_strength(text),
                       'sentiment': self.sentiment_analyzer.polarity_scores(text)['compound']}
                for name, text in profiles_dict.items()}
    def _extract_skills(self, text):
        text_lower = text.lower()
        return {category: min(sum(text_lower.count(kw) for kw in keywords) * 15, 100)
                for category, keywords in self.config['skill_keywords'].items()}
    def _extract_experience(self, text):
        matches = re.findall(r'(\d+)\+?\s*years?', text.lower())
        return max([int(m) for m in matches]) if matches else self.config['experience_default']
    def _extract_network_strength(self, text):
        score = sum(text.lower().count(k) for k in self.config['network_keywords'])
        return min(score * self.config['network_multiplier'], 100)

class HeuristicSynergyCalculator:
    def __init__(self, config): self.config = config['synergy_calculation']
    def calculate(self, analysis_data):
        names = list(analysis_data.keys())
        skill_vectors = np.array([list(analysis_data[n]['skills'].values()) for n in names])
        comp_matrix = 1 - cosine_similarity(skill_vectors)
        experiences = [analysis_data[n]['experience_years'] for n in names]
        exp_matrix = np.array([[np.exp(-0.1 * abs(exp_i - exp_j)) if i != j else 0 for j, exp_j in enumerate(experiences)] for i, exp_i in enumerate(experiences)])
        networks = [analysis_data[n]['network_strength'] for n in names]
        net_matrix = np.array([[(net_i * net_j) / 10000.0 if i != j else 0 for j, net_j in enumerate(networks)] for i, net_i in enumerate(networks)])
        w = self.config['weights']
        total_synergy = (w['complementarity'] * comp_matrix + w['experience'] * exp_matrix + w['network'] * net_matrix)
        backgrounds = self.config['cultural_backgrounds']
        for i, n1 in enumerate(names):
            for j, n2 in enumerate(names):
                if backgrounds.get(n1) != backgrounds.get(n2): total_synergy[i, j] *= self.config['diversity_bonus']
        np.fill_diagonal(total_synergy, 0)
        return {'total_synergy': total_synergy, 'skill_scores': {n: analysis_data[n]['skills'] for n in names}}

class DynamicSystemModel:
    def __init__(self, config): self.config = config['dynamic_model']
    def simulate(self, synergy_matrix):
        c = self.config
        beta = synergy_matrix * c['beta_multiplier']
        time_axis = np.linspace(0, c['time_horizon'], c['time_horizon'] * 4)
        solution = odeint(self._system_dynamics, c['initial_state'], time_axis, args=(c['alpha'], beta, c['K']))
        return time_axis, solution
    def _system_dynamics(self, state, t, alpha, beta, K):
        x = np.array(state)
        return (alpha * x + np.dot(beta, x)) * (1 - x / K)

class PredictionModel:
    def __init__(self, config): self.config = config['prediction']
    def run_all(self, growth_solution, time_axis, synergy_matrix):
        forecast = self._forecast_business_metrics(growth_solution, time_axis, synergy_matrix)
        monte_carlo = self._monte_carlo_simulation(growth_solution, time_axis, synergy_matrix)
        roi = self._calculate_roi_metrics(forecast)
        return {'forecast': forecast, 'monte_carlo': monte_carlo, 'roi': roi}
    def _forecast_business_metrics(self, growth_data, time_axis, synergy_matrix):
        perf_data = [{'time_months': time_axis[t], 'revenue_growth_rate': (np.sum(growth_data[t]) + np.sum(synergy_matrix * np.outer(growth_data[t], growth_data[t]))/2) * 20} for t in range(len(time_axis))]
        return pd.DataFrame(perf_data)
    def _monte_carlo_simulation(self, growth_data, time_axis, synergy_matrix):
        n_sim = self.config['monte_carlo_simulations']
        results = []
        for _ in range(n_sim):
            noise = np.random.normal(1, 0.1, growth_data.shape)
            sim_growth = growth_data * noise
            final_perf = np.sum(sim_growth[-1])
            results.append({'final_performance': final_perf, 'success': 1 if final_perf > np.sum(growth_data[0]) * 1.5 else 0})
        return pd.DataFrame(results)
    def _calculate_roi_metrics(self, forecast_df):
        monthly_revenue = forecast_df['revenue_growth_rate'] * self.config['revenue_scale_factor']
        cumulative_revenue = monthly_revenue.cumsum()
        cost = self.config['investment_cost']
        roi_over_time = (cumulative_revenue - cost) / cost * 100
        payback_period = next((t for t, rev in zip(forecast_df['time_months'], cumulative_revenue) if rev >= cost), None)
        discount_rate = self.config['npv_discount_rate'] / 12
        npv = np.sum(monthly_revenue / (1 + discount_rate)**np.arange(len(monthly_revenue))) - cost
        return {'roi_over_time': roi_over_time, 'payback_period_months': payback_period, 'npv': npv}

class GameTheoryAnalyzer:
    def __init__(self, config): self.config = config['game_theory']
    def calculate_shapley(self, analysis_data, synergy_matrix):
        names, n_players = list(analysis_data.keys()), len(analysis_data)
        shapley_values = np.zeros(n_players)
        for i in range(n_players):
            for coalition_size in range(n_players):
                for coalition in combinations(range(n_players), coalition_size):
                    if i in coalition: continue
                    marginal_contrib = self._get_coalition_value(list(coalition) + [i], analysis_data, synergy_matrix) - self._get_coalition_value(list(coalition), analysis_data, synergy_matrix)
                    weight = (math.factorial(coalition_size) * math.factorial(n_players - coalition_size - 1)) / math.factorial(n_players)
                    shapley_values[i] += weight * marginal_contrib
        return shapley_values
    def _get_coalition_value(self, indices, analysis_data, synergy_matrix):
        if not indices: return 0
        names = list(analysis_data.keys())
        base_val = sum(analysis_data[names[i]]['experience_years'] + analysis_data[names[i]]['network_strength'] / 10.0 for i in indices)
        synergy_val = sum(synergy_matrix[i, j] for i in indices for j in indices if i != j)
        return base_val + synergy_val * self.config['synergy_value_multiplier']

class VisualizationGenerator:
    def __init__(self, config):
        self.config = config
    def generate_all_figures(self, results):
        figures, names = {}, self.config['data']['display_names']
        # スキルレーダー
        fig_radar = go.Figure()
        for name in names:
            fig_radar.add_trace(go.Scatterpolar(r=list(results['synergy']['skill_scores'][name].values()), theta=list(self.config['profile_analysis']['skill_keywords'].keys()), fill='toself', name=name))
        fig_radar.update_layout(title='V4: Skill Profile (Keyword-based)')
        figures['skill_radar'] = fig_radar
        # シナジーヒートマップ
        figures['synergy_heatmap'] = px.imshow(results['synergy']['total_synergy'], x=names, y=names, text_auto='.3f', title='V4: Overall Synergy Matrix')
        # 成長軌道
        df_g = pd.DataFrame(results['simulation_solution'], columns=names); df_g['Time (Months)'] = results['simulation_time_axis']
        figures['growth_trajectory'] = px.line(df_g, x='Time (Months)', y=names, title='V4: Dynamic Growth Simulation (Stable)')
        # モンテカルロ
        figures['monte_carlo'] = px.histogram(results['predictions']['monte_carlo'], x='final_performance', nbins=50, title='V4: Monte Carlo Simulation of Final Performance')
        # ROI
        df_roi = pd.DataFrame({'ROI (%)': results['predictions']['roi']['roi_over_time'], 'Time (Months)': results['predictions']['forecast']['time_months']})
        figures['roi_analysis'] = px.line(df_roi, x='Time (Months)', y='ROI (%)', title='V4: ROI Over Time')
        # シャプレー値
        figures['shapley_values'] = px.bar(x=names, y=results['shapley_values'], text_auto='.2f', title='V4: Value Contribution (Shapley Values)')
        return figures

# ===============================================================
# STEP 5: 実行パイプラインの定義と実行
# ===============================================================
class AnalysisPipeline:
    def __init__(self, config):
        self.config = config
        self.data_loader = DataLoader(config)
        self.profile_analyzer = KeywordProfileAnalyzer(config)
        self.synergy_calculator = HeuristicSynergyCalculator(config)
        self.dynamic_model = DynamicSystemModel(config)
        self.prediction_model = PredictionModel(config)
        self.game_theory_analyzer = GameTheoryAnalyzer(config)
        self.viz_generator = VisualizationGenerator(config)
        self.run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    def run(self):
        print("--- パイプライン実行開始 V4 (総合安定版) ---")
        output_folder = os.path.join(self.config['output']['base_path'], f"{self.config['output']['folder_prefix']}{self.run_timestamp}")
        os.makedirs(output_folder, exist_ok=True)
        print(f"✅ 出力先フォルダ準備完了: '{output_folder}' (同期のため5秒待機)")
        time.sleep(5)

        profiles = self.data_loader.load_profiles()
        analysis_data = self.profile_analyzer.analyze(profiles)
        synergy_results = self.synergy_calculator.calculate(analysis_data)
        time_axis, solution = self.dynamic_model.simulate(synergy_results['total_synergy'])
        predictions = self.prediction_model.run_all(solution, time_axis, synergy_results['total_synergy'])
        shapley_values = self.game_theory_analyzer.calculate_shapley(analysis_data, synergy_results['total_synergy'])

        results = {'config': self.config, 'analysis': analysis_data, 'synergy': synergy_results,
                   'simulation_time_axis': time_axis, 'simulation_solution': solution,
                   'predictions': predictions, 'shapley_values': shapley_values}

        results['figures'] = self.viz_generator.generate_all_figures(results)
        self._save_artifacts(results, output_folder)
        print("\n--- ✅ パイプライン実行完了 ---")
        return results

    def _save_artifacts(self, results, output_folder):
        # 統合HTMLダッシュボード
        html_content = self._create_combined_html(results['figures'])
        with open(os.path.join(output_folder, "dashboard.html"), 'w', encoding='utf-8') as f: f.write(html_content)
        print(f"✅ 統合ダッシュボードを保存しました。")
        # 詳細レポート
        report_content = self._generate_report(results)
        with open(os.path.join(output_folder, "report.md"), 'w', encoding='utf-8') as f: f.write(report_content)
        print(f"✅ 詳細レポートを保存しました。")

    def _create_combined_html(self, figures):
        html = "<html><head><title>Synergy Analysis Dashboard V4</title><script src='https://cdn.plot.ly/plotly-latest.min.js'></script></head><body>"
        for name, fig in figures.items():
            html += f"<h2>{fig.layout.title.text}</h2><div id='{name}'>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>"
        html += "</body></html>"
        return html

    def _generate_report(self, results):
        names = self.config['data']['display_names']
        roi_final = results['predictions']['roi']['roi_over_time'].iloc[-1]
        payback = results['predictions']['roi']['payback_period_months']
        success_prob = results['predictions']['monte_carlo']['success'].mean() * 100

        report = f"# Synergy Analysis Report V4 ({self.run_timestamp})\n\n"
        report += f"## Executive Summary\n"
        report += f"- **Success Probability (Monte Carlo):** {success_prob:.1f}%\n"
        report += f"- **Final ROI:** {roi_final:.1f}%\n"
        report += f"- **Payback Period:** {payback if payback else 'N/A'} months\n\n"
        report += f"## Shapley Values (Value Contribution)\n"
        for name, val in zip(names, results['shapley_values']):
            report += f"- {name}: {val:.2f}\n"
        return report

# パイプラインを実行
pipeline = AnalysisPipeline(CONFIG)
results = pipeline.run()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.3 MB/s eta 0:00:00
✅ ライブラリのインストールが完了しました。
✅ ライブラリのインポートが完了しました。
✅ 分析パラメータの定義が完了しました。
--- パイプライン実行開始 V4 (総合安定版) ---
✅ 出力先フォルダ準備完了: '/content/drive/MyDrive/synergy_full_analysis_20250620_221804' (同期のため5秒待機)
Mounted at /content/drive
✅ 統合ダッシュボードを保存しました。
✅ 詳細レポートを保存しました。

--- ✅ パイプライン実行完了 ---
